In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)


Exploratory data analysis (EDA) is a crucial step in any data analysis project, as it helps to understand the characteristics of the data, detect patterns, and identify potential issues. Here are some important EDA methods for tabular data:

Descriptive statistics: Descriptive statistics such as mean, median, mode, variance, and standard deviation provide a summary of the data and help to identify outliers and potential errors.

Histograms: Histograms are a useful way to visualize the distribution of a single variable. They display the frequency of data points in bins or intervals.

Box plots: Box plots summarize the distribution of a variable by showing the median, quartiles, and potential outliers.

Scatter plots: Scatter plots display the relationship between two variables. They can help to identify patterns, trends, and potential outliers.

Correlation analysis: Correlation analysis is used to determine the strength and direction of the relationship between two variables. It can help to identify which variables are potentially important in a regression model.

Heatmaps: Heatmaps display the correlation between multiple variables using color-coded cells. They are useful for identifying patterns and relationships among multiple variables.

Principal component analysis (PCA): PCA is a technique used to reduce the dimensionality of the data by transforming correlated variables into a smaller set of uncorrelated variables. It is useful for identifying patterns and relationships in large datasets.

Clustering: Clustering is a technique used to group similar data points together based on their similarities in multiple variables. It can help to identify patterns and relationships in the data.

Missing value analysis: Missing value analysis is used to identify the missing values in the data and determine how they may impact the analysis. It can help to decide whether to impute missing values or exclude them from the analysis.

When working with tabular data involving fraud, there are some specific exploratory data analysis (EDA) techniques that are particularly important. Here are some EDA methods for tabular data involving fraud:

Outlier Analysis: Outliers are often indicative of fraudulent activity and can be identified by exploring data using techniques such as box plots or scatter plots. In addition, statistical tests can be used to identify potential outliers.

Anomaly Detection: Anomaly detection techniques are used to identify unusual patterns in data that may indicate fraudulent activity. Common anomaly detection techniques include clustering, decision trees, and support vector machines.

Time Series Analysis: Fraudulent activity often occurs over time, and time series analysis can be used to identify patterns and trends in data that may indicate fraudulent behavior. This can include looking at the frequency of transactions, the amount of money involved, and other variables.

Data Visualization: Data visualization is a powerful tool for identifying patterns and relationships in data. Techniques such as heatmaps, scatter plots, and histograms can be used to explore data and identify potential fraud.

Correlation Analysis: Correlation analysis can be used to identify the strength and direction of relationships between variables in a data set. This can be particularly useful in identifying patterns that may indicate fraudulent activity.

things to do:

grab the dataset

General Information:
check info() describe() and head()

Column Types
- print df.dtypes() to get a series of each column and their respective datatypes.
- Ensure that each column's datatype correctly represents the values inside: eg. if its integers, make sure theyre ints/floats, not strings.
- For each column, determine whether or not the listed datatype matches the intention of the datatype
    For each column:

        If it is type object (string), list it as a categorical column and verify point 1 on 'Column Analysis'. If this is the case, convert the column into a integer or float datatype

        If it is type int, ensure it isn't a fake int (eg its a flag: [0,1] or a count [1,5]). may need to create a threshold variable (say number of unique values less than 20) to differentiate the continuous variables and the fake continuous variables (use .nunique() for a column on the dataframe). Include these predictors in a num_but_cat column

        If it is type float, ensure some values are real floats and not just integers in type float (in that case you will need to check for any num_but_cat columns)

        
Missing Values AND Non-sensical Values
- Missing Values: For each column, calculate the ratio of blanks to non-blanks or a percentage, and return the total length of the blanks in the column. If the ratio/percentage is quite high, then it may be better to just remove the variable/column altogether. Can put this all together into a pandas series. Generally we will leave missing values in, as the tree method implemented in Python can deal with them

- Non-sensical Values: For each categorical column, it may be wise to get a set of unique values for each column, and make sure each value makes sense in the context of the column's name/description. For example, 'XNA' in the gender column does not make sense. In this case, it maybe useful to make these nonsense values blank or remove the rows from the dataframe itself.

Graphs

- We'll have to remove all the missing values from our eda plots, since they'll bug out if we do not remove them.

- For continuous predictors, we generally use the kde plot. Locate values in the dataframe for our column where the target == 1 and where target == 0, plot them both on the same graph on the same axis and stuff. We can also plot them all on the same graph to see its distribution

- For categorical predictors, we generally use the histogram. Locate values in the dataframe where target == 0 and target == 1, plot both histograms, and plot them all together in a third one.

Value Counts

- For categorical variables, we can plot a small matrix of each category and their respective counts and percentage of the data. We can see if any specific category dominates the majority of the values, and if so, we can remove the column (say 98% for apartments etc..) DONE

Transformations

- For continuous variables, we can see if the data is skewed in any particular direction using the df.skew() function. If applicable, we can apply a log transformation on the data to see if we can normalize the set of data. For continuous variables with unbounded range (like income for example), its important to scale the variables using some sort of transformation for better accuracy of the model.

Correlation Matrix

- Plot the correlation matrix and remove columns that are collinear with other columns


Visual Analysis:

- Can remove columns that you feel are not necessary to include in the model. eg: the flag documents


AUCROC

PRAUC

LiftChart

ks statistic

Explanability
SHAP VALUES

Individual Feature importance

for predictor in predictor_list:
    model target from predictor
    calculate pr auc

rank pr auc, sort descending, get top 100 or some amount


xgboost built in feature importance


ridge feature selection (would have to transform all variables to normality first)

In [2]:
os.getcwd()
%cd data

c:\Users\Julian\Documents\projs\home-credit-default-risk\data


Helper Functions

In [3]:
def show_cat_col_values(df, cols = None):
    """ 
    df: DataFrame
    cols: list/Dataframe

    Displays the categorical options for each categorical variable
    """
    if cols:
        for col in cols:
            ratios = []
            values = pd.DataFrame(df[col].value_counts())
            for value in values[col].values:
                total = np.sum(values[col].values)
                ratios.append(value/total)
            
            values['Ratio'] = ratios

            print(values.sort_values(by = 'Ratio', ascending = False))
            print("\n")
    else:
        cats = df.select_dtypes(include = ['object'])
        for col in cats:
            ratios = []
            values = pd.DataFrame(df[col].value_counts())
            for value in values[col].values:
                total = np.sum(values[col].values)
                ratios.append(value/total)

            values['Ratio'] = ratios

            print(values.sort_values(by = 'Ratio', ascending = False))
            print("\n")

def show_missing_values(df):
    """ 
    df: DataFrame

    Prints the missing perentage of every column
    """
    na_dict = {}
    for col in df.columns:
        na = df[col].isna().sum()
        total = len(df[col])

        na_dict[col] = f"{round(100*na/total,2)}%"

    print(pd.Series(na_dict).sort_values(ascending = False))

def remove_cat_col_values(df, threshold, cols = None):
    """
    df: DataFrame
    threshold: (0,1) float
    cols: list/DataFrame

    If one single value dominates the entire column, then we remove the column entirely, as there is too little variation in the column values
    """
    assert 0 <= threshold <= 1, "threshold must be between 0 and 1"
    count = 0
    if cols:
        for col in cols:
            values = pd.DataFrame(df[col].value_counts())
            for value in values[col].values:
                total = np.sum(values[col].values)
                if value/total >= threshold:
                    count += 1
                    df = df.drop([col], axis = 1)

    cats = df.select_dtypes(include = ['object'])
    for col in cats:
        values = pd.DataFrame(df[col].value_counts())
        for value in values[col].values:
            total = np.sum(values[col].values)
            if value/total >= threshold:
                count += 1
                df = df.drop([col], axis = 1)

    print(f"{count} columns have been removed")
    return df


def remove_missing_values(df, threshold):
    """ 
    df: DataFrame
    threshold: (0,1) float

    Removes the columns that have a missing value percentage greater than the threshold
    """
    assert 0 <= threshold <= 1, "threshold must be between 0 and 1"
    count = 0
    for col in df.columns:
        na = df[col].isna().sum()
        total = len(df[col])

        if na/total > threshold:
            df = df.drop([col], axis = 1)
            count += 1
    print(f"{count} columns have been removed")
    
    return df

def remove_bad_rows(df, col, value):
    """
    df: DataFrame
    col: string
    value: string/float/int

    Removes the selected rows from the dataset
    """
    rows = df.loc[df[col] == value]
    
    df = df.drop(rows.index, axis = 0)
    print(f"A total of {len(rows)} have been removed")
    return df

def show_ratio(df, target, col):
    """
    df: DataFrame
    target: [0,1]
    col: string

    Shows the ratio of target values between each cateogorical value. For example, if Female has a ratio of 13, then for every 13 non-defaults, there will be one female that defaults
    Ratios that are not consistent between all categorical values may indicate that there will be some impact on the model
    """
    dict_ = {}
    for value in set(df[col].values):
        
        target_1 = df.loc[(df[target] == 1) & (df[col] == value)]
        target_0 = df.loc[(df[target] == 0) & (df[col] == value)]

        if len(target_1) == len(target_0) == 0:
            dict_[value] = -1
        elif len(target_1) == 0:
            dict_[value] = -2
        else:
            dict_[value] = len(target_0)/len(target_1)
    
    series = pd.Series(dict_).sort_values(ascending = False)
    series = series.replace({-1: "no values found", -2: f"{value} has all target values of 0"})
    print(col)
    print(series)
    print("\n")

def show_ratio_by_target(df, target, cols = None):
    """
    df: DataFrame
    target: [0,1] float
    cols = string/list/DataFrame

    This can take either a list, string, or all of the object dataframes in the dataframe
    Shows the ratio of target values between each cateogorical value. For example, if Female has a ratio of 13, then for every 13 non-defaults, there will be one female that defaults
    Ratios that are not consistent between all categorical values may indicate that there will be some impact on the model
    """
    if isinstance(cols, str):
        show_ratio(df, target, cols)

    elif isinstance(cols, list):
        for col in cols:
            if col != target:
                show_ratio(df, target, col)

    elif isinstance(df, pd.DataFrame):
        for col in df.select_dtypes('object'):
            if col != target:
                show_ratio(df, target, col)

    else:
        return "Invalid type for parameter col"


def show_unique_keys(df):
    """ 
    df: DataFrame

    Shows the unique keys in the dataframe
    """
    for col in df.columns:
        print(df, df[col].is_unique)

In [4]:
def cat_but_num(df):
    """ 
    df: DataFrame

    Try to encode any float but string columns as floats correctly
    """
    cat_but_num = []
    for col in df.select_dtypes("object"):
        try:
            df[col] = df[col].astype(float)
            cat_but_num.append(col)
        except:
            #print(f"This column {col} cannot be converted into a float")
            continue

    return cat_but_num
        

def num_but_cat(df, threshold):
    """ 
    df: DataFrame
    threshold: (0,1) float

    Encode any numerical columns as categorical if the number of unique values is less than the threshold
    """
    num_but_cat = []
    for col in df.select_dtypes(['number']):

        if df[col].nunique() < threshold:
            num_but_cat.append(col)

    return num_but_cat

def grab_col_types(df, threshold):
    """ 
    df: DataFrame
    threshold: (0,1) float

    Obtain the numerical and categorical columns and return them as lists
    """
    cat_but_num_list = cat_but_num(df)

    num_but_cat_list = num_but_cat(df, threshold)

    numerical_cols = [col for col in df.select_dtypes(['number']) if col not in num_but_cat_list]
    cat_cols = [col for col in df.select_dtypes(['object']) if col not in cat_but_num_list]

    numerical_cols = numerical_cols + cat_but_num_list
    cat_cols = cat_cols + num_but_cat_list

    return numerical_cols, cat_cols


In [5]:
def plot_kde(df, col):
    na = df[col].isna().sum()
    total = len(df[col])
    missing = round(100*na/total, 2)

    if 'TARGET' in df.columns:
        plt.figure(figsize = (12, 6))
        plt.subplot(1,2,1)
        sns.kdeplot(df.loc[df['TARGET'] == 0, col], label = 'TARGET == 0')
        sns.kdeplot(df.loc[df['TARGET'] == 1, col], label = 'TARGET == 1')
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.legend()

        plt.subplot(1,2,2)
        sns.kdeplot(df[col].values, label = 'All')
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.legend()

        plt.show()
    else:
        plt.subplot(1,1,1)
        sns.kdeplot(df[col].values, label = 'All')
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.legend()

        plt.show()

def kde_plots(df, cols):
    if isinstance(cols, list):
        for col in cols:
            plot_kde(df,col)
    else:
        plot_kde(df, col)

def plot_hist(df, col):
    na = df[col].isna().sum()
    total = len(df[col])
    missing = round(100*na/total, 2)
    
    if 'TARGET' in df.columns:
        plt.figure(figsize = (12, 6))
        plt.subplot(1,2,1)
        sns.histplot(df.loc[df['TARGET'] == 0, col], color='orange')
        sns.histplot(df.loc[df['TARGET'] == 1, col], color='blue')
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.legend(labels=['TARGET == 0', 'TARGET == 1'])

        plt.subplot(1,2,2)
        sns.histplot(df[col].values)
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.show()

    else:
        plt.subplot(1,2,1)
        sns.histplot(df[col].values)
        plt.xlabel(col); plt.ylabel('Density'); plt.title(f"{col} Distribution, {missing}% Missing")
        plt.show()

def hist_plots(df, cols):
    if isinstance(cols, list):
        for col in cols:
            plot_hist(df, col)
    else:
        plot_hist(df, col)


In [6]:
def show_corr(df):
    f = plt.figure(figsize=(106, 106))
    plt.matshow(df.corr(), fignum=f.number)
    plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=45)
    plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title('Correlation Matrix', fontsize=16)


def remove_corr(df, threshold):
    m = df.corr().where(np.triu(np.ones(df.corr().shape), k=1).astype(bool))
    col = [col for col in m.columns if any(m[col].values > threshold)]
    print(f"{len(col)} columns have been removed")
    print("The following columns will be removed")
    for column in col:
        print(column)
    df.drop(col, axis = 1, inplace = True)

Load the Dataset and display any introductory information

In [7]:
raw_train = pd.read_csv(os.getcwd()+"\\train_agg.csv")

In [8]:
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 473 entries, SK_ID_CURR to install_payments_AMT_PAYMENT_mean
dtypes: float64(416), int64(41), object(16)
memory usage: 1.1+ GB


In [9]:
raw_train.describe()

c:\Users\Julian\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
c:\Users\Julian\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,bureau_DAYS_CREDIT_mean,bureau_CREDIT_DAY_OVERDUE_mean,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_max,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CNT_CREDIT_PROLONG_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_var,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_OVERDUE_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_var,bureau_CREDIT_TYPE_Another type of loan_sum,bureau_CREDIT_TYPE_Car loan_sum,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Loan for business development_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_TYPE_Loan for the purchase of equipment_sum,bureau_CREDIT_TYPE_Loan for working capital replenishment_sum,bureau_CREDIT_TYPE_Microloan_sum,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Mortgage_sum,bureau_CREDIT_TYPE_Real estate loan_sum,bureau_CREDIT_TYPE_Unknown type of loan_sum,credit_card_MONTHS_BALANCE_max,credit_card_MONTHS_BALANCE_mean,credit_card_MONTHS_BALANCE_sum,credit_card_MONTHS_BALANCE_median,credit_card_MONTHS_BALANCE_std,credit_card_AMT_BALANCE_max,credit_card_AMT_BALANCE_mean,credit_card_AMT_BALANCE_sum,credit_card_AMT_BALANCE_median,credit_card_AMT_BALANCE_std,credit_card_AMT_CREDIT_LIMIT_ACTUAL_max,credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean,credit_card_AMT_CREDIT_LIMIT_ACTUAL_sum,credit_card_AMT_CREDIT_LIMIT_ACTUAL_median,credit_card_AMT_CREDIT_LIMIT_ACTUAL_std,credit_card_AMT_DRAWINGS_ATM_CURRENT_max,credit_card_AMT_DRAWINGS_ATM_CURRENT_mean,credit_card_AMT_DRAWINGS_ATM_CURRENT_sum,credit_card_AMT_DRAWINGS_ATM_CURRENT_median,credit_card_AMT_DRAWINGS_ATM_CURRENT_std,credit_card_AMT_DRAWINGS_CURRENT_max,credit_card_AMT_DRAWINGS_CURRENT_mean,credit_card_AMT_DRAWINGS_CURRENT_sum,credit_card_AMT_DRAWINGS_CURRENT_median,credit_card_AMT_DRAWINGS_CURRENT_std

In [10]:
raw_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,bureau_DAYS_CREDIT_mean,bureau_CREDIT_DAY_OVERDUE_mean,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_max,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_CNT_CREDIT_PROLONG_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_var,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_OVERDUE_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_var,bureau_CREDIT_TYPE_Another type of loan_sum,bureau_CREDIT_TYPE_Car loan_sum,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Loan for business development_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_TYPE_Loan for the purchase of equipment_sum,bureau_CREDIT_TYPE_Loan for working capital replenishment_sum,bureau_CREDIT_TYPE_Microloan_sum,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Mortgage_sum,bureau_CREDIT_TYPE_Real estate loan_sum,bureau_CREDIT_TYPE_Unknown type of loan_sum,credit_card_MONTHS_BALANCE_max,credit_card_MONTHS_BALANCE_mean,credit_card_MONTHS_BALANCE_sum,credit_card_MONTHS_BALANCE_median,credit_card_MONTHS_BALANCE_std,credit_card_AMT_BALANCE_max,credit_card_AMT_BALANCE_mean,credit_card_AMT_BALANCE_sum,credit_card_AMT_BALANCE_median,credit_card_AMT_BALANCE_std,credit_card_AMT_CREDIT_LIMIT_ACTUAL_max,credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean,credit_card_AMT_CREDIT_LIMIT_ACTUAL_sum,credit_card_AMT_CREDIT_LIMIT_ACTUAL_median,credit_card_AMT_CREDIT_LIMIT_ACTUAL_std,credit_card_AMT_DRAWINGS_ATM_CURRENT_max,credit_card_AMT_DRAWINGS_ATM_CURRENT_mean,credit_card_AMT_DRAWINGS_ATM_C

In [11]:
raw_train.dtypes

SK_ID_CURR                                                                    int64
TARGET                                                                        int64
NAME_CONTRACT_TYPE                                                           object
CODE_GENDER                                                                  object
FLAG_OWN_CAR                                                                 object
FLAG_OWN_REALTY                                                              object
CNT_CHILDREN                                                                  int64
AMT_INCOME_TOTAL                                                            float64
AMT_CREDIT                                                                  float64
AMT_ANNUITY                                                                 float64
AMT_GOODS_PRICE                                                             float64
NAME_TYPE_SUITE                                                             

In [12]:
print(f"The number of rows in the dataset is {len(raw_train)}")
print(f"The percentage of defaulted loans in this dataset is: {sum(raw_train['TARGET'].values)/len(raw_train['TARGET'].values)}")

The number of rows in the dataset is 307511
The percentage of defaulted loans in this dataset is: 0.08072881945686496


In [13]:
print("Columns in this dataset")
for col in raw_train.columns:
    print(col)

Columns in this dataset
SK_ID_CURR
TARGET
NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
OCCUPATION_TYPE
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
WEEKDAY_APPR_PROCESS_START
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
ORGANIZATION_TYPE
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG

In [14]:
print(f"The number of blank values in this dataset is: {np.sum(raw_train.isnull().sum())}")

#This indicates that we have many blank values throughout the dataset and it would be wise to keep the rows that contain them or else we will have too little data to work with

The number of blank values in this dataset is: 47902380


We choose to omit the flag documents columns

In [15]:
drop_cols = [col for col in raw_train.columns if 'XNA' in col]

drop_flag_cols = [f'FLAG_DOCUMENT_{i}' for i in range(2,22)]

In [16]:
raw_train_dropped = raw_train.drop(drop_cols, axis = 1)

raw_train_dropped = raw_train_dropped.drop(drop_flag_cols, axis = 1)

In [17]:
print(raw_train.shape)
print(raw_train_dropped.shape)

(307511, 473)
(307511, 444)


Feature Engineering

In [18]:
raw_train['PAYMENT_PER_PERIOD'] = raw_train['AMT_ANNUITY'] / raw_train['AMT_CREDIT']
raw_train['AVG_EXTERNAL_CREDIT_SCORE'] = (raw_train['EXT_SOURCE_1']+raw_train['EXT_SOURCE_2']+raw_train['EXT_SOURCE_3'])/3
raw_train['DEBT_RATIO'] = raw_train['AMT_CREDIT']/raw_train['AMT_INCOME_TOTAL']
raw_train['ANNUITY_INCOME_RATIO'] = raw_train['AMT_ANNUITY']/raw_train['AMT_INCOME_TOTAL']

In [19]:
show_ratio_by_target(raw_train, 'TARGET')

NAME_CONTRACT_TYPE
Revolving loans    17.253741
Cash loans         10.981913
dtype: float64


CODE_GENDER
F                         13.287085
M                          8.860066
XNA    F has all target values of 0
dtype: object


FLAG_OWN_CAR
Y    12.805042
N    10.764392
dtype: float64


FLAG_OWN_REALTY
Y    11.560325
N    11.012114
dtype: float64


NAME_TYPE_SUITE
Children                 12.556017
Family                   12.342971
Spouse, partner          11.703911
Unaccompanied            11.220386
Group of people          10.782609
Other_A                  10.394737
Other_B                   9.172414
NaN                no values found
dtype: object


NAME_INCOME_TYPE
Pensioner                                              17.565392
State servant                                          16.376301
Commercial associate                                   12.361381
Working                                                 9.429191
Unemployed                                                

In [20]:
raw_train['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
raw_train['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace = True)

bad_rows = [['CODE_GENDER', 'XNA'], ['NAME_FAMILY_STATUS', 'Unknown']]

for row in bad_rows:
    raw_train_dropped = remove_bad_rows(raw_train_dropped, row[0], row[1])

A total of 4 have been removed
A total of 2 have been removed


In [21]:
print(raw_train_dropped.shape)

(307505, 444)


In [22]:
show_missing_values(raw_train_dropped)

prev_app_DAYS_FIRST_DRAWING_std                                             99.95%
credit_card_AVG_DRAWINGS_OTHER_CURRENT_std                                  99.35%
credit_card_AVG_DRAWINGS_OTHER_CURRENT_max                                  98.15%
credit_card_AVG_DRAWINGS_OTHER_CURRENT_mean                                 98.15%
credit_card_AVG_DRAWINGS_OTHER_CURRENT_median                               98.15%
credit_card_AVG_DRAWINGS_ATM_CURRENT_std                                    93.37%
credit_card_AVG_DRAWINGS_POS_CURRENT_std                                    90.97%
credit_card_AVG_DRAWINGS_ATM_CURRENT_mean                                   90.44%
credit_card_AVG_DRAWINGS_POS_CURRENT_max                                    88.88%
credit_card_AVG_DRAWINGS_POS_CURRENT_mean                                   88.88%
credit_card_AVG_DRAWINGS_POS_CURRENT_median                                 88.88%
credit_card_AVG_DRAWINGS_CURRENT_std                                         84.6%
prev

In [23]:
num, cat = grab_col_types(raw_train_dropped, 10)

print("Numerical Columns:")
for col in num:
    print(col)
print("Categorical Columns:")
for col in cat:
    print(col)


Numerical Columns:
SK_ID_CURR
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
CNT_FAM_MEMBERS
HOUR_APPR_PROCESS_START
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONLIVINGAREA_MODE
APARTMENTS_MEDI
BASEMENTAREA_MEDI
YEARS_BEGINEXPLUATATION_MEDI
YEARS_BUILD_MEDI
COMMONAREA_MEDI
ELEVATORS_MEDI
ENTRANCES_MEDI
FLOORSMAX_MEDI
FLOORSMIN_MEDI
LANDAREA_MEDI
LIVINGAPARTMENTS_MEDI
LIVINGAREA_MEDI
NONLIVINGAPARTMENTS_MEDI
N

In [24]:
show_cat_col_values(raw_train_dropped, cat)

                 NAME_CONTRACT_TYPE     Ratio
Cash loans                   278232  0.904805
Revolving loans               29273  0.095195


   CODE_GENDER     Ratio
F       202447  0.658354
M       105058  0.341646


   FLAG_OWN_CAR     Ratio
N        202920  0.659892
Y        104585  0.340108


   FLAG_OWN_REALTY     Ratio
Y           213306  0.693667
N            94199  0.306333


                 NAME_TYPE_SUITE     Ratio
Unaccompanied             248523  0.811596
Family                     40148  0.131110
Spouse, partner            11370  0.037131
Children                    3267  0.010669
Other_B                     1770  0.005780
Other_A                      866  0.002828
Group of people              271  0.000885


                      NAME_INCOME_TYPE     Ratio
Working                         158771  0.516320
Commercial associate             71614  0.232887
Pensioner                        55362  0.180036
State servant                    21703  0.070578
Unemployed             

In [25]:
raw_train_dropped = remove_cat_col_values(raw_train_dropped, 0.95, cat)

60 columns have been removed


In [26]:
print(raw_train_dropped.shape)

(307505, 384)


Encoding categorical variables

In [27]:
train_cats = pd.get_dummies(raw_train_dropped)

In [28]:
print(train_cats.shape)

(307505, 503)


In [29]:
#show_corr(train_cats)

In [30]:
remove_corr(train_cats, 0.85)

167 columns have been removed
The following columns will be removed
AMT_GOODS_PRICE
CNT_FAM_MEMBERS
REGION_RATING_CLIENT_W_CITY
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE
NONLIVINGAPARTMENTS_MODE
NONLIVINGAREA_MODE
APARTMENTS_MEDI
BASEMENTAREA_MEDI
YEARS_BEGINEXPLUATATION_MEDI
YEARS_BUILD_MEDI
COMMONAREA_MEDI
ELEVATORS_MEDI
ENTRANCES_MEDI
FLOORSMAX_MEDI
FLOORSMIN_MEDI
LANDAREA_MEDI
LIVINGAPARTMENTS_MEDI
LIVINGAREA_MEDI
NONLIVINGAPARTMENTS_MEDI
NONLIVINGAREA_MEDI
TOTALAREA_MODE
OBS_60_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
bureau_AMT_ANNUITY_var
credit_card_MONTHS_BALANCE_sum
credit_card_MONTHS_BALANCE_median
credit_card_AMT_BALANCE_mean
credit_card_AMT_BALANCE_median
credit_card_AMT_BALANCE_std
credit_card_AMT_CREDIT_LIMIT_ACTUAL_mean
credit_card_AMT_CREDIT_LIMIT_ACTUAL_median
credit_

In [31]:
print(train_cats.shape)

(307505, 336)


In [32]:
train_num, train_cat = grab_col_types(train_cats, 0)

In [33]:
#kde_plots(train_cats, train_num)

In [34]:
#hist_plots(train_cats, train_cat)

In [35]:
train_cats.replace([np.inf, -np.inf], np.nan, inplace=True)

Saving the dataset

In [36]:
train_cats.to_csv("data.csv", index = False)

In [37]:
data_columns = pd.DataFrame(columns = train_cats.columns)
data_columns.to_csv("data_columns.csv", index = False)